In [1]:
!pip install ktrain==0.23.2    #Latest model is not working....

     |████████████████████████████████| 25.3MB 125kB/s 
     |████████████████████████████████| 983kB 51.4MB/s 
     |████████████████████████████████| 266kB 37.4MB/s 
     |████████████████████████████████| 1.9MB 43.0MB/s 
     |████████████████████████████████| 471kB 52.2MB/s 
     |████████████████████████████████| 3.2MB 52.4MB/s 
     |████████████████████████████████| 890kB 50.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.23.2-cp37-none-any.whl size=25272008 sha256=041a61d8ccff6f441cf5e8c44cf71b1bca81a289d295f2aff58a58268855243e
  Stored in directory: /root/.cache/pip/wheels/fe/4a/b1/aff404c4e0893ca1c0b64781d0298b7ff6de94d5117a0e7d5c
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp37-none-any.whl size=34144 sha256=fa2f5741d3a0c630e4a80a490d1bb409dec34040f29f3adf4917405a43e7eb7f
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl siz

In [7]:
import tensorflow as tf
import pandas as pd
import ktrain

In [3]:
!git clone https://github.com/LawrenceDuan/IMDb-Review-Analysis   #dataset

Cloning into 'IMDb-Review-Analysis'...
remote: Enumerating objects: 26, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (26/26), done.


In [5]:
from ktrain import text

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
data_train1=pd.read_csv('/content/IMDb-Review-Analysis/IMDb_Reviews.csv')

In [14]:
data_train

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0
...,...,...
34995,This has got to be the worst horror movie I ha...,0
34996,"""De Dominee"" is based on the life of a real du...",0
34997,This was usually producer Alexander Korda's ad...,1
34998,There is a complete copy now available at the ...,1


In [13]:
data_train=data_train1.iloc[:35000,:]
data_test=data_train1.iloc[35000:,:]

In [15]:
(X_train,y_train),(X_test,y_test),preprocess=text.texts_from_df(train_df=data_train,text_column='review',label_columns='sentiment',maxlen=400,preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [19]:
X_train[0].shape

(31500, 400)

In [20]:
X_train

[array([[ 101, 1045, 5292, ...,    0,    0,    0],
        [ 101, 2092, 1045, ...,    0,    0,    0],
        [ 101, 2096, 3666, ...,    0,    0,    0],
        ...,
        [ 101, 1037, 2442, ...,    0,    0,    0],
        [ 101, 1045, 2442, ..., 2091, 2045,  102],
        [ 101, 2793, 2243, ...,    0,    0,    0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [21]:
model=text.text_classifier(name='bert',train_data=(X_train,y_train),preproc=preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [22]:
#To Get Learning rate
learner=ktrain.get_learner(model=model,train_data=(X_train,y_train),val_data=(X_test,y_test),batch_size=6,) 

In [23]:
'''
learner.lr_find()
learner.lr_plot()
'''
#Takes so much time to run....Gives optimal learning rate

#For this problem optimal Learning rate is 2e-5

'\nlearner.lr_find()\nlearner.lr_plot()\n'

In [30]:
learner.fit_onecycle(lr=2e-5,epochs=2)

In [29]:
predictor=ktrain.get_predictor(learner.model,preprocess)



predictor.save('/content/drive/MyDrive/IMDB_review_classification/predictor')

['not_sentiment', 'sentiment', 'not_sentiment', 'sentiment']


In [40]:
predictor_load=ktrain.load_predictor('/content/drive/MyDrive/IMDB_review_classification/predictor')

In [45]:
data=['Movie was bull shit!!!!I want my money back','Movie was beautiful.Very feel good movie','Worst movie I have ever seen','great plot greatt acting great performances great script','Not a sexual movie and good one']

c=0
for i in predictor_load.predict(data):
  if i=='not_sentiment':
    print(data[c],":- is a Negative review")
  else:
    print(data[c],":- is a Positive review")
  c=c+1

Movie was bull shit!!!!I want my money back :- is a Negative review
Movie was beautiful.Very feel good movie :- is a Positive review
Worst movie I have ever seen :- is a Negative review
great plot greatt acting great performances great script :- is a Positive review
Not a sexual movie and good one :- is a Positive review


In [48]:
#!zip -r /content/drive/predictor.zip/content/drive/MyDrive/IMDB_review_classification/predictor